<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Обзор-и-предобработка-данных" data-toc-modified-id="Обзор-и-предобработка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Обзор и предобработка данных</a></span></li><li><span><a href="#Проверка-гипотез" data-toc-modified-id="Проверка-гипотез-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Проверка гипотез</a></span><ul class="toc-item"><li><span><a href="#Сравнение-поведения-пользователей-двух-столиц" data-toc-modified-id="Сравнение-поведения-пользователей-двух-столиц-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Сравнение поведения пользователей двух столиц</a></span></li><li><span><a href="#Музыка-в-начале-и-в-конце-недели" data-toc-modified-id="Музыка-в-начале-и-в-конце-недели-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Музыка в начале и в конце недели</a></span></li><li><span><a href="#Жанровые-предпочтения-в-Москве-и-Петербурге" data-toc-modified-id="Жанровые-предпочтения-в-Москве-и-Петербурге-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Жанровые предпочтения в Москве и Петербурге</a></span></li></ul></li><li><span><a href="#Итоги-исследования" data-toc-modified-id="Итоги-исследования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Итоги исследования</a></span></li></ul></div>

# Сравнение музыкальных вкусов Москвы и Санкт-Петербурга (Яндекс.Музыка)

Сравнение Москвы и Петербурга окружено мифами. Например:
 * Москва — мегаполис, подчинённый жёсткому ритму рабочей недели;
 * Петербург — культурная столица, со своими вкусами.

На данных Яндекс Музыки сравним поведение пользователей двух столиц.

**Цель исследования** — проверить три гипотезы:
1. Активность пользователей зависит от дня недели. Причём в Москве и Петербурге это проявляется по-разному.
2. В понедельник утром в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города. 
3. Москва и Петербург предпочитают разные жанры музыки. В Москве чаще слушают поп-музыку, в Петербурге — русский рэп.

**Ход исследования**

Данные о поведении пользователей находятся в `yandex_music_project.csv`.

Поверим данне на ошибки и оценим их влияние на исследование. На этапе предобработки рассмотрим возможности исправить самые критичные ошибки данных.
 
Таким образом, исследование пройдёт в три этапа:
 1. Обзор данных.
 2. Предобработка данных.
 3. Проверка гипотез.

## Обзор и предобработка данных

Составьте первое представление о данных Яндекс Музыки.


In [1]:
# Импортируем библиотеки
import pandas as pd 

In [6]:
# Откроем файл с данными
df = pd.read_csv('yandex_music_project.csv')

In [9]:
# Функция первичного обзора данных
def review(data):
    display(data.head(5))
    print(data.info())
    print('Пропуски:', data.isna().sum())
    print('Явные дубликаты:')
    if data.duplicated().sum() > 0:
        print(data.duplicated().sum())
    else:
        print('Не найдено')

In [10]:
# Используем функцию для первичного обзора df
review(df)

,userID,Track,artist,genre,City,time,Day;;
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday;;
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday;;
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday;;
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday;;
4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday;;


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65215 entries, 0 to 65214
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65215 non-null  object
 1   Track     63256 non-null  object
 2   artist    57330 non-null  object
 3   genre     63292 non-null  object
 4     City    64487 non-null  object
 5   time      64487 non-null  object
 6   Day;;     64485 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB
None
Пропуски:   userID       0
Track       1959
artist      7885
genre       1923
  City       728
time         728
Day;;        730
dtype: int64
Явные дубликаты:
3927


Итак, в таблице семь столбцов. Тип данных во всех столбцах — `object`.

Согласно документации к данным:
* `userID` — идентификатор пользователя;
* `Track` — название трека;  
* `artist` — имя исполнителя;
* `genre` — название жанра;
* `City` — город пользователя;
* `time` — время начала прослушивания;
* `Day` — день недели.

Можно выделить следующие проблемы с данными:
- Названия столбцов подлежат изменению на змеиный регистр. 
- Тип данных времени не соответствует требуемому, его поменяем на datetime.
- Столбец со днем недели (Day) имеет ненужные символы.
- В данных есть пропущенные значения и явные дубликаты.

In [18]:
# Переименование столбцов
df = df.rename(columns = {'  userID':'user_id', 'Track':'track', '  City  ':'city', 'Day;;':'day'})

In [12]:
# Заменим пропущенные значения в столбцах track, artist и genre на строку 'unknown'
columns_to_replace = ['track', 'artist', 'genre'] 

for column in columns_to_replace:
    df[column] = df[column].fillna('unknown')

In [13]:
# Удаление явных дубликатов
df = df.drop_duplicates()

In [26]:
# Функция проверки наличия неявных дубликатов
def duplicates_check(data):
    print(data.sort_values().unique())

In [28]:
# Проверим данные на наличие неявных дубликатов в столбцах genre, city и day
columns_to_check = ['genre', 'city', 'day'] 

for column in columns_to_check:
    duplicates_check(df[column])

['Moscow' 'acid' 'acoustic' 'action' 'adult' 'africa' 'afrikaans'
 'alternative' 'alternativepunk' 'ambient' 'americana' 'animated' 'anime'
 'arabesk' 'arabic' 'arena' 'argentinetango' 'art' 'audiobook' 'author'
 'avantgarde' 'axé' 'baile' 'balkan' 'beats' 'bigroom' 'black' 'bluegrass'
 'blues' 'bollywood' 'bossa' 'brazilian' 'breakbeat' 'breaks' 'broadway'
 'cantautori' 'cantopop' 'canzone' 'caribbean' 'caucasian' 'celtic'
 'chamber' 'chanson' 'children' 'chill' 'chinese' 'choral' 'christian'
 'christmas' 'classical' 'classicmetal' 'club' 'colombian' 'comedy'
 'conjazz' 'contemporary' 'country' 'cuban' 'dance' 'dancehall' 'dancepop'
 'dark' 'death' 'deep' 'deutschrock' 'deutschspr' 'dirty' 'disco' 'dnb'
 'documentary' 'downbeat' 'downtempo' 'drum' 'dub' 'dubstep' 'eastern'
 'easy' 'electronic' 'electropop' 'emo' 'entehno' 'epicmetal' 'estrada'
 'ethnic' 'eurofolk' 'european' 'experimental' 'extrememetal' 'fado'
 'fairytail' 'film' 'fitness' 'flamenco' 'folk' 'folklore' 'folkmetal'
 'f

Есть неявные дубликаты в столбцах genre и day:
* *hip*,
* *hop*,
* *hip-hop*.


* *Friday;*
* *Friday;;*
* *Monday*
* *Monday;*
* *Monday;;*
* *Wednesday;*
* *Wednesday;;*

Заменим в столбце с жанрами `hip`, `hop` и `hip-hop` на `hiphop`, а в  столбце с названиями дней недели уберем точки с запятой.

В столбце city есть некорректные значения '08:40:52' '09:28:57', их удалим.

In [15]:
# Устранение неявных дубликатов в столбце genre
genre_to_replace = ['hip', 'hop', 'hip-hop']
df['genre'] = df['genre'].replace(genre_to_replace,'hiphop')

In [22]:
# Устранение неявных дубликатов в столбце day
df['day'] = df['day'].replace(to_replace =';', value = '', regex = True)

In [37]:
# Устранение некорректных значений в столбце city
df = df[(df['city']!='08:40:52')&(df['city']!='09:28:57')]

**Выводы**

В ходе предобработки внесены изменения:

- Исправлены нарушения в стиле заголовков,
- Пропущенные значения в столбцах track, artist и genre заменили на `'unknown'`.
- Устранены дубликаты — явные и неявные (в столбцах genre и day).
- Удалены некорректыне значения из столбца city

## Проверка гипотез

### Сравнение поведения пользователей двух столиц

Первая гипотеза утверждает, что пользователи по-разному слушают музыку в Москве и Санкт-Петербурге. Проверим это предположение по данным о трёх днях недели — понедельнике, среде и пятнице. Для этого:

* Разделим пользователей Москвы и Санкт-Петербурга.
* Сравним, сколько треков послушала каждая группа пользователей в понедельник, среду и пятницу.


In [39]:
# Подсчёт прослушиваний в каждом городе
print(df.groupby('city')['user_id'].count())

city
Moscow              42332
Saint-Petersburg    18358
Name: user_id, dtype: int64


В Москве прослушиваний больше, чем в Петербурге. Из этого не следует, что московские пользователи чаще слушают музыку. Просто самих пользователей в Москве больше.

In [40]:
# Подсчёт прослушиваний в каждый из трёх дней
print(df.groupby('day')['user_id'].count())

day
Friday       21621
Monday       21183
Wednesday    17886
Name: user_id, dtype: int64


В среднем пользователи из двух городов менее активны по средам. Но картина может измениться, если рассмотреть каждый город в отдельности.

In [41]:
# Функция расчета прослушивания для заданного дня и города
def number_tracks(day, city):
    track_list=df[(df['day']==day)&(df['city']==city)]
    track_list_count=track_list['user_id'].count()
    return track_list_count

In [42]:
# количество прослушиваний в Москве по понедельникам
number_tracks('Monday', 'Moscow')

15614

In [43]:
# количество прослушиваний в Санкт-Петербурге по понедельникам
number_tracks('Monday', 'Saint-Petersburg') 

5569

In [44]:
# количество прослушиваний в Москве по средам
number_tracks('Wednesday', 'Moscow')

10939

In [45]:
# количество прослушиваний в Санкт-Петербурге по средам
number_tracks('Wednesday', 'Saint-Petersburg') 

6947

In [46]:
# количество прослушиваний в Москве по пятницам
number_tracks('Friday', 'Moscow')

15779

In [47]:
# количество прослушиваний в Санкт-Петербурге по пятницам
number_tracks('Friday', 'Saint-Petersburg')

5842

Создадим c помощью конструктора pd.DataFrame таблицу, где:

- названия колонок — ['city', 'monday', 'wednesday', 'friday'];
- данные — результаты, которые мы получили с помощью number_tracks.

In [48]:
# Таблица с результатами
data = [['Moscow', 15740, 11056, 15945],
       ['Saint-Petersburg', 5614, 7003, 5895]]
columns = ['city','monday','wednesday','friday']
info = pd.DataFrame(data = data, columns = columns)
info 

,city,monday,wednesday,friday
0,Moscow,15740,11056,15945
1,Saint-Petersburg,5614,7003,5895


**Выводы**

Данные показывают разницу поведения пользователей:

- В Москве пик прослушиваний приходится на понедельник и пятницу, а в среду заметен спад.
- В Петербурге, наоборот, больше слушают музыку по средам. Активность в понедельник и пятницу здесь почти в равной мере уступает среде.

Значит, данные говорят в пользу первой гипотезы.

### Музыка в начале и в конце недели

Согласно второй гипотезе, утром в понедельник в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города.

In [50]:
# Сохраним данные по городам в отдельных переменных
moscow_general = df[df['city'] == 'Moscow']
spb_general = df[df['city'] == 'Saint-Petersburg']

In [53]:
# Функция вывода топ-10 жанров тех треков, которые прослушивали в указанный день, в промежутке между двумя отметками времени
def genre_weekday(df, day, time1, time2):
    genre_df = df[(df['day'] == day)&(df['time'] > time1)&(df['time'] < time2)]
    genre_df_count=genre_df.groupby('genre')['user_id'].count()
    genre_df_count=genre_df_count.sort_values(ascending=False)
    print(genre_df_count.head(10))

Cравним результаты функции genre_weekday() для Москвы и Санкт-Петербурга в понедельник утром (с 7:00 до 11:00) и в пятницу вечером (с 17:00 до 23:00):

In [54]:
# Предпочтения городов в понедельник утром (с 7:00 до 11:00)
genre_weekday(moscow_general, 'Monday', '07:00', '11:00')
genre_weekday(spb_general, 'Monday', '07:00', '11:00')

genre
pop            775
dance          549
electronic     480
rock           474
hiphop         286
ruspop         186
world          180
rusrap         175
alternative    164
unknown        161
Name: user_id, dtype: int64
genre
pop            216
dance          182
rock           162
electronic     147
hiphop          80
ruspop          63
alternative     58
rusrap          55
jazz            43
classical       36
Name: user_id, dtype: int64


In [55]:
# Предпочтения городов в пятницу вечером (с 17:00 до 23:00)
genre_weekday(moscow_general, 'Friday', '17:00', '23:00')
genre_weekday(spb_general, 'Friday', '17:00', '23:00')

genre
pop            706
rock           516
dance          493
electronic     482
hiphop         273
world          207
ruspop         170
alternative    163
rusrap         142
classical      135
Name: user_id, dtype: int64
genre
pop            253
electronic     216
rock           215
dance          210
hiphop          97
alternative     63
jazz            61
rusrap          59
classical       53
world           53
Name: user_id, dtype: int64


**Выводы**

Если сравнить топ-10 жанров в понедельник утром, можно сделать такие выводы:

1. В Москве и Петербурге слушают похожую музыку. Единственное отличие — в московский рейтинг вошёл жанр “world”, а в петербургский — джаз и классика.

2. В Москве пропущенных значений оказалось так много, что значение `'unknown'` заняло десятое место среди самых популярных жанров. Значит, пропущенные значения занимают существенную долю в данных и угрожают достоверности исследования.

Вечер пятницы не меняет эту картину. Некоторые жанры поднимаются немного выше, другие спускаются, но в целом топ-10 остаётся тем же самым.

Таким образом, вторая гипотеза подтвердилась лишь частично:
* Пользователи слушают похожую музыку в начале недели и в конце.
* Разница между Москвой и Петербургом не слишком выражена. В Москве чаще слушают русскую популярную музыку, в Петербурге — джаз.

Однако пропуски в данных ставят под сомнение этот результат. В Москве их так много, что рейтинг топ-10 мог бы выглядеть иначе, если бы не утерянные  данные о жанрах.

### Жанровые предпочтения в Москве и Петербурге

Гипотеза: Петербург — столица рэпа, музыку этого жанра там слушают чаще, чем в Москве.  А Москва — город контрастов, в котором, тем не менее, преобладает поп-музыка.

In [58]:
# Сгруппируем таблицы с данными по городам по жанру 
moscow_genres = moscow_general.groupby('genre')['genre'].count().sort_values(ascending=False)
spb_genres = spb_general.groupby('genre')['genre'].count().sort_values(ascending=False)

In [59]:
print(moscow_genres.head(10))#
print(spb_genres.head(10))#

genre
pop            5840
dance          4432
rock           3956
electronic     3784
hiphop         2095
classical      1427
world          1423
alternative    1377
ruspop         1368
rusrap         1160
Name: genre, dtype: int64
genre
pop            2410
dance          1931
rock           1875
electronic     1734
hiphop          960
alternative     647
classical       582
rusrap          564
ruspop          535
world           513
Name: genre, dtype: int64


**Выводы**

Гипотеза частично подтвердилась:
* Поп-музыка — самый популярный жанр в Москве, как и предполагала гипотеза. Более того, в топ-10 жанров встречается близкий жанр — русская популярная музыка.
* Вопреки ожиданиям, рэп одинаково популярен в Москве и Петербурге. 


## Итоги исследования

Мы проверили три гипотезы и установили:

**Гипотеза 1. Активность пользователей зависит от дня недели. Причём в Москве и Петербурге это проявляется по-разному.**
Исследование показало, что день недели по-разному влияет на активность пользователей в Москве и Петербурге.
В Москве пик прослушиваний приходится на понедельник и пятницу, а в среду заметен спад.
В Петербурге, наоборот, больше слушают музыку по средам. Активность в понедельник и пятницу здесь почти в равной мере уступает среде.

_Первая гипотеза полностью подтвердилась._

**Гипотеза 2. В понедельник утром в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города.**
Проверка гипотезы показала, что музыкальные предпочтения не сильно меняются в течение недели — будь то Москва или Петербург. Небольшие различия заметны в начале недели, по утрам понедельника:
* в Москве слушают музыку жанра “world”,
* в Петербурге — джаз и классику.
Вечер пятницы не меняет эту картину. Некоторые жанры поднимаются немного выше, другие спускаются, но в целом топ-10 остаётся тем же самым.

Таким образом, _вторая гипотеза подтвердилась лишь отчасти_. Этот результат мог оказаться иным, если бы не пропуски в данных.

**Гипотеза 3. Москва и Петербург предпочитают разные жанры музыки. В Москве чаще слушают поп-музыку, в Петербурге — русский рэп.**
Оказалочь, что во вкусах пользователей Москвы и Петербурга больше общего чем различий. Вопреки ожиданиям, предпочтения жанров в Петербурге напоминают московские:
* Поп-музыка — самый популярный жанр в Москве, как и предполагала гипотеза. Более того, в топ-10 жанров встречается близкий жанр — русская популярная музыка.
* Вопреки ожиданиям, рэп одинаково популярен в Москве и Петербурге.

_Третья гипотеза не подтвердилась_. Если различия в предпочтениях и существуют, на основной массе пользователей они незаметны.